## Обучение LLaMa генерировать заголовки новостей

<div>
<center>
<img src="text_title_example.png" width="900"/>
</center>
</div>

### 1. Установим и импортируем необходимые библиотеки

In [ ]:
!pip install -r requirements.txt

**get_peft_model** - это функция, которая будет использоваться для добавления адаптеров по конфигурации в **LoraConfig** перед дообучением.
**prepare_model_for_int8_training** - передаем модель в эту функцию, если хотим обучать модель в int8.

In [ ]:
import pickle
import numpy as np
import torch
import transformers
from transformers import LlamaTokenizer, LlamaForCausalLM, BitsAndBytesConfig

from peft import (
    LoraConfig,
    get_peft_model
)

### 2. Инициализация основной модели

In [ ]:
# Зададим, на каком устройстве будем обучать модель (GPU или CPU).
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Название в Hugging Face Hub предобученной модели, чекпоинты которой мы будем использовать
# Если используем LLaMa1
#BASE_MODEL = "decapoda-research/llama-7b-hf"

# Для LLaMa 2:
# BASE_MODEL = "meta-llama/Llama-2-7b-hf"

# Если используем закрытую модель, такую как LLaMa 2:
# В функцию from_pretrained нужно добавлять token от личного кабинета Hugging Face
# tokenizer = LlamaTokenizer.from_pretrained(BASE_MODEL, use_auth_token=token)

# Мы уже скачали веса в локальную директорию torch_trnsf_models/llama-2-7b-hf:
BASE_MODEL = "torch_trnsf_models/llama-2-7b-hf"

# Инициализируем токенизатор для модели LLaMa
tokenizer = LlamaTokenizer.from_pretrained(BASE_MODEL)

# задаем токен для padding, то есть добавления в те последовательности из батча, которые короче,
# чем максимальная длина последовательности, чтобы все последовательности в итоге были одной длины
tokenizer.pad_token_id = 0

# нули для padding будем добавлять слева
tokenizer.padding_side = "left"

In [ ]:
# Инициализируем основную модель
"""
model = LlamaForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
"""

In [ ]:
# Инициализируем основную модель
# Если мы хотим обучать модель в int4 для уменьшения требуемой видеопамяти

model = LlamaForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    load_in_4bit=True,
    device_map="auto",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4', # квантизация модели в тип normal float 4
    )
)

[1] Dettmers, T., Pagnoni, A., Holtzman, A., & Zettlemoyer, L. (2023). Qlora: Efficient finetuning of quantized llms. arXiv preprint arXiv:2305.14314.

Число с плавающей точкой представляется в памяти в виде $(-1)^З * M * 2^{э - смещение}$

<div>
<center>
<img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/blog/bitsandbytes/FP8-scheme.png" width="900"/>
</center>
</div>

**Сравнение потребления видеопамяти для разных конфигураций.**

| Конфигурация | VRAM |
| :--- | :--- |
| 32bit | 24.8 Gb |
| 8bit | 22Gb |
| 4bit, BitsAndBytesConfig | 15.6 Gb |

### 3. Загрузка и предобработка данных

Считываем датасет из файла

In [ ]:
from sklearn.model_selection import train_test_split

with open("news_titles_dataset.pickle", 'rb') as inp:
    dataset = pickle.load(inp)

print("dataset length", len(dataset))
print("title:", dataset[5]["title"])
print("text:", dataset[5]["text"])

**Функция для получения строки с промптом** по инструкции, входному тексту и тексту, который должна сгенерировать модель:

In [ ]:
INSTRUCTION = "Сгенерируй заголовок к данному тексту."

# Максимальная длина последовательности токенов на входе трансформера (если пос-ть длинее, обрезаем до CUTOFF_LEN)
CUTOFF_LEN = 500

def generate_prompt(sample):
    # Также для разделения частей промпта можно использовать специальные токены начала и конца сегмента <s> и </s>
    prompt = f"{INSTRUCTION}\nТекст:\n{sample['text']}\nЗаголовок: "
    full_prompt = f"{INSTRUCTION}\nТекст:\n{sample['text']}\nЗаголовок: {sample['title']}"

    # Если длина full_prompt больше, чем CUTOFF_LEN, удалим несколько последних предложений текста,
    # пока длина не станет меньше, чем CUTOFF_LEN
    if len(tokenizer(full_prompt)["input_ids"]) > CUTOFF_LEN:
        sentences = sample['text'].split(". ")   # делим текст на предложения
        while True:
            sentences = sentences[:-1]
            text = ". ".join(sentences)
            prompt = f"{INSTRUCTION}\nТекст:\n{text}\nЗаголовок: "
            full_prompt = f"{INSTRUCTION}\nТекст:\n{text}\nЗаголовок: {sample['title']}"
            if len(tokenizer(full_prompt)["input_ids"]) < CUTOFF_LEN:
                break
    return prompt, full_prompt

**Функция для токенизации промпта**:

In [ ]:
def tokenize(prompt, full_prompt, add_eos_token=True):
    result = tokenizer(
        full_prompt,
        padding=False,
        return_tensors=None,
    )
    if (
            result["input_ids"][-1] != tokenizer.eos_token_id
            and len(result["input_ids"]) < CUTOFF_LEN
            and add_eos_token
    ):
        # если в конце пос-ти нет специального токена, мы его добавляем
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    prompt_len = len(tokenizer(prompt)["input_ids"])
    labels = result["input_ids"].copy()
    labels = [-100 for _ in range(prompt_len)] + labels[prompt_len:]
    result["labels"] = labels

    return result

В функции **generate_and_tokenize_prompt** по примеру из датасета, переданному на вход, мы сначала получаем промпт, а потом его токенизируем. Дальше эту функцию применим для предобработки всех примеров из датасета.

In [ ]:
def generate_and_tokenize_prompt(sample):
    prompt, full_prompt = generate_prompt(sample)
    tokenized_full_prompt = tokenize(prompt, full_prompt)
    return tokenized_full_prompt

In [ ]:
# Разделим выборку на обучающую и валидационную
train_data, test_data = train_test_split(dataset, test_size=0.1)

# Предобработка данных (получение промпта для каждого примера из датасета и последующая токенизация)
train_data = list(map(generate_and_tokenize_prompt, train_data))
test_data = list(map(generate_and_tokenize_prompt, test_data))

In [ ]:
# data_collator нужен для формирования батча (padding, сборка элементов батча в один тензор,
# конвертация массивов numpy или списков в тензоры torch.LongTensor)
data_collator = transformers.DataCollatorForSeq2Seq(
    tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True, label_pad_token_id=-100
)

### 4. Добавляем в LLaMa адаптеры

Добавляем в трансформер (LLaMa), который будет заморожен во время обучения, обучаемые слои (адаптеры). То есть основные слои LLaMa останутся неизменные, обучаться будут только адаптеры. У добавляемых адаптеров значительно меньше параметров, чем у основной модели, за счет чего удается значительно сократить требуемый объем вычислительных ресурсов.

<div>
<center>
<img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/peft/lora_diagram.png" width="900"/>
</center>
</div>

In [ ]:
# Размерность матриц адаптеров
# К примеру, если исходная матрица весов 4096 x 4096, то матрицы, которые мы добавляем,
# имеют размерность 4096 х LORA_R и LORA_R х 4096.
LORA_R = 8

# После умножения на матрицу весов адаптеров компоненты вектора делим на LORA_R и умножаем на LORA_ALPHA
LORA_ALPHA = 16
LORA_DROPOUT = 0.05

# В какие слои трансформера будем добавлять адаптеры, в данном случае - в матрицы в слоях self-attention
# для вычисления query и key.
LORA_TARGET_MODULES = [
    "q_proj",
    "v_proj",
]

# Делаем объект конфигурации по параметрам адаптеров
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=LORA_TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)

**Добавляем в трансформер адаптеры по параметрам, которые были переданы в LoraConfig.**

In [ ]:
model = get_peft_model(model, config)

# Выведем информацию об обучаемых весах модели.
model.print_trainable_parameters()

### 5. Задаем гиперпараметры обучения

In [ ]:
BATCH_SIZE = 4
TRAIN_EPOCHS = 3
MICRO_BATCH_SIZE = 2
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
LEARNING_RATE = 3e-4

# директория, в которую будем сохранять чекпоинты модели
OUTPUT_DIR = "news_checkpoints"

training_arguments = transformers.TrainingArguments(
    per_device_train_batch_size=MICRO_BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    warmup_steps=100,
    max_steps=2000,
    num_train_epochs=TRAIN_EPOCHS,
    learning_rate=LEARNING_RATE,
    fp16=True,
    logging_steps=100,
    optim="adamw_torch",
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=100,
    save_steps=100,
    output_dir=OUTPUT_DIR,
    save_total_limit=3,
    load_best_model_at_end=True,
    report_to="none"
)

### 6. Запускаем обучение

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    args=training_arguments,
    data_collator=data_collator
)

In [ ]:
model.config.use_cache = False

# компиляция модели (для оптимизации обучения)
model = torch.compile(model)

In [ ]:
# Обучение
trainer.train()

**Сохраняем чекпоинты обученной модели**

In [ ]:
model.save_pretrained(OUTPUT_DIR)